In [1]:
#Installed packages required to extract data from wikipedia
pip install lxml html5lib beautifulsoup4

     |████████████████████████████████| 5.8MB 18.0MB/s eta 0:00:018MB 18.0MB/s eta 0:00:01
     |████████████████████████████████| 112kB 32.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Before starting, let's download all the dependencies that we will need.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

Extracting tables from wikipedia link 

In [4]:
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = read_html(page, attrs={'class':'wikitable'})
print("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


In [5]:
wikitables[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


wikitables[0] is assigned to dataframe df 

In [6]:
df = wikitables[0]

The dataframe now consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Only those cells are processed that have an assigned borough. Cells with Not assigned borough are removed as shown below;

In [8]:
df = df[df.Borough != 'Not assigned']

In [9]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


now dataframe df is assigned to dataframe df_toronto with indexes reset as shown below;

In [10]:
df_toronto = df.reset_index(drop=True)

now the borough column does not contain not assigned cells

In [11]:
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. In order to validate if any Neighborhood cell with not assigned exist following code is written below;

In [12]:
empty_neighborhood = df_toronto[df_toronto['Neighborhood'].isna()].shape[0]
print("the cells which are assigned borough but not neighborhood:", empty_neighborhood)

the cells which are assigned borough but not neighborhood: 0


More than one neighborhood can exist on a single postal code and borough, so in order to address this issue groupby function is used below based on postal code & borough

In [13]:
df_toronto=df_toronto.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [14]:
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


dataframe df_toronto is assigned to dataframe df_torontonew

In [15]:
df_torontonew = df_toronto

In [16]:
df_torontonew.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [55]:
print ("the dataframe has following rows & columns:", df_torontonew.shape)

the dataframe has following rows & columns: (103, 3)


for geocoding provided csv file has been used 

In [18]:
df_coords=pd.read_csv('Geospatial_Coordinates.csv')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


data frame df_torontonew & df_coords have been merged to create a single dataframe df_torontomerge  in order to utilize foursquare location data 

In [19]:
df_torontomerge=pd.merge(df_torontonew, df_coords, how='inner', on='Postal Code')

In [20]:
df_torontomerge.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
df_torontomerge.shape

(103, 5)